In [2]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from PIL import Image
import tensorflow
import keras
from keras.datasets import mnist
from keras.layers import Input, Dense, Flatten, MaxPooling2D, MaxPooling1D, Conv2D, Reshape, Dropout
from keras.layers import BatchNormalization, ZeroPadding2D
from keras.models import Model, Sequential
from keras import regularizers
import numpy as np
import matplotlib.pyplot as plt
import csv
import zipfile
from skimage.transform import resize
from sklearn.metrics import classification_report

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython: from IPython import display

In [6]:
class DQN(keras.Model):
    def __init__(self, img_height, img_width):
        super().__init__()
        input_shape = img_height*img_width*3

        self.model = Sequential()
        self.model.add(Reshape(input_shape, input_shape=input_shape))
        self.model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
        self.model.add(Conv2D(64, (3, 3), activation='relu'))
        self.model.add(MaxPooling2D(pool_size=(2, 2)))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dropout(0.2))
        self.model.add(BatchNormalization())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(2, activation='softmax'))

        self.model.compile(loss='catergorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])
        
    def Fit(self,train_images,train_labels,batch_size,test_images,test_labels):
        self.model.fit(train_images, train_labels,
        batch_size=batch_size,
        epochs=8,
        validation_data=(test_images, test_labels))

In [7]:
Experience = namedtuple(
    'Experience',
    ('state', 'action', 'next_state', 'reward')
)

e = Experience(2,3,1,4) 
e, e[1]

(Experience(state=2, action=3, next_state=1, reward=4), 3)

In [ ]:
class ReplayMemory():
    def __init__(self, capacity):
        self.capacity = capacity #Max Memory
        self.memory = []
        self.push_count = 0
        
    def push(self, experience):
        if len(self.memory) < self.capacity:
            self.memory.append(experience) #Append Experiences
        else:
            self.memory[self.push_count % self.capacity] = experience #% is used to grab remainder after division
        self.push_count += 1
        
    def sample(self, batch_size): #sample random batch from replay memory
        return random.sample(self.memory, batch_size)
    
    def can_provide_sample(self, batch_size): #to ensure we can provide the required batchsize and there is sufficient experiences
        return len(self.memory) >= batch_size